Atividades
===

Essas atividade são relativas a aulas de extração de atributos e selecão de atributos. 
- Link para video: https://drive.google.com/file/d/13LLQGLt7QsjKshBXgUdJ4RVOT8N252cU/view
- Link para apresentação: https://docs.google.com/presentation/d/1wctFgQe7TSlBEypZbVQqqsfrnpSgOeXPSO-mCJ6jIz4


> Lembre de criar uma `virtualenv` com os `requirements.txt` do repositório.

> Lembre de criar uma `kernel` do jupyter para seus desenvolvimentos

> **MANTENHA** essa arquivo no local padrão do repositório. Detro da pasta **feature_extraction_and_feature_selection**.

> Enviei apenas o seu notebook para correção. Não é necessário enviar nenhum outro arquivo. Siga o padrão `Atividades-<NOME-DO-ALUNO>.ipynb`

# Feature Extraction

## 1. Realizar a extração de atributos utilzando as técnicas Fourier e HOS nas bases de dados

- Dados de falhas em aerogeradores

As orientações abaixo devem ser seguidas antes de iniciar os trabalhos

> Todo os arquivos nomeados com `v000_ ... .csv` são referentes a essa base.

> A classe referente ao dado está representada por uma TAG no nome do arquivo da seguinte forma: `v000_NORMAL_ ... .csv` é o dado referente a classe de funcionamento normal do gerador. A TAG `v000_SC_LI_LVL3_ ... .csv` é a classe referente a falha tipo 1 e continua. Não se preocupem com a ordem de enumeração das classes nesse momento.

> Há 5 arquivos na pasta.

> Utilizar as colunas a `Current_R`, `Current_S`, `Current_T` para realizar extração de atributos. Perceba que cada coluna é referente a uma sinal de corrente elétrica, portanto cada um são representados vetores de dimensões $1x50000$.

> Antes de realizar a extração de atributos deve-se dividir cada sinal em 10 partes de tamanho $1x5000$. A ideia aqui é aumentar o número de amostra da base de dados por 10. Pense que cada parte é um recorte do sinal, e ao serem concatenadas retornarão o sinal original.

# Imports  

### Importing feature extraction module  

In [37]:
!git clone https://github.com/navarmn/feature_extraction_signal.git

fatal: destination path 'feature_extraction_signal' already exists and is not an empty directory.


In [38]:
from feature_extraction_signal.src import feature_extraction

### importing pandas and others tools

In [39]:
import pandas as pd
import numpy as np
import os.path
from matplotlib import pyplot as plt

### Carregue a base

In [40]:
dataset_folder = os.path.join('.', 'datasets')
DATAFILES = [
    'v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv',
    'v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv',
    'v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv'
]

In [41]:
'''
We are going to have an array which each element is a dataframe with the full signal
'''

signals = [pd.read_csv(os.path.join(dataset_folder, DATAFILES[x])).iloc[:, :3] for x in range(len(DATAFILES))]

### Split signals

#### This code below will split the dataframe in an array with N dataframes

In [42]:
def splitDataframe(df, slices=10):
    #collecting number of rows and columns
    df_rows, df_columns = df.shape
    
    #defining the slice
    slice = int(df_rows/slices)
    
    #defining variables to start spliting the dataframe
    prev_slice, current_slice = 0, 0
    slices_array = [] #array that will receive the slices
    for x in range(slices):
        #update the prev and current slice variables and append the slice to slices_array 
        prev_slice = current_slice
        current_slice+=slice
        slices_array.append(df.iloc[prev_slice:current_slice])
    return slices_array    

### Joining a sliced dataframe

In [43]:
def joinDataframe(array_of_dfs):
    dataframe = pd.DataFrame([])
    for df in array_of_dfs:
        dataframe = dataframe.append(df, ignore_index=True)
    return dataframe

In [44]:
'''
Now we are going to split each signal, that by now are dataframes. So at the end of the process we
will have an array which each element it will be an another array with the sliced signals.

[[slice1DF, slice2DF, ..., slice10DF], ...]

'''

splited_signals = []
for signal in signals:
    splited_signals.append(splitDataframe(signal))

    
#selecting just the three fisrt columns of the splited dataframe
#array_of_dfs = [df.iloc[:, :3] for df in array_of_dfs]

### a) Comparativo da média do sinal e sua distribuição

- Realize o calculo da média e desvio padrão em cada um dos sinais e exiba o resultado em um dataframe. Discuta suas conclusões.

**help**: utilize as funcões nativas do numpy ou do pandas. Lembre que o sinal é uma senoide, qual a média em uma senoide simétrica?

In [60]:
def mean_and_std(array_of_dfs, how='slices'):
    '''
    Two types of use:
    full: Use the full signal to calculate the mean and the standard deviation.
    
    slices: Use the slices to calculate the mean and the standard deviation and return the result
    of each slice as a Dataframe
    '''

    #dataframes that will be merged
    df_mean, df_std = pd.DataFrame([]),pd.DataFrame([])
    
    #dataframe that will be returned
    return_df = pd.DataFrame([])
    
    #joins the dataframe
    dataframe = joinDataframe(array_of_dfs)

    #getting columns names and changing them
    cols_names = dataframe.columns
    mean_names, std_names = ['mean_'+name for name in cols_names],['std_'+name for name in cols_names]
    
    if how == 'full':
        
        #calculating the mean and the standard deviation of the dataframe
        serie_mean, serie_std = dataframe.mean(), dataframe.std()
        
        #changing the index of the series
        serie_mean.index,serie_std.index = mean_names,std_names
        
        #returns the mean series concatenated with the std series
        return pd.concat([serie_mean, serie_std], join='outer', sort=False)

    if how == 'slices':
        #deleting dataframe variable that will not be used anymore
        del dataframe
        
        for df in array_of_dfs:

            #calculating the mean and the standard deviation of the current dataframe(df)
            serie_mean, serie_std = df.mean(), df.std()

            #changing the index of the series
            serie_mean.index,serie_std.index = mean_names,std_names

            #appending the serie_mean and the serie_std to df_mean and df_std, respectively
            df_mean, df_std = df_mean.append(serie_mean, ignore_index=True), df_std.append(serie_std, ignore_index=True, sort=False)
    
        #Merging the two dataframes (df_mean and df_std)
        return_df = df_mean.merge(df_std, left_index=True, right_index=True, how='outer')
        
            
    return return_df

for i, splited_signal in enumerate(splited_signals):
    print('\n'+DATAFILES[i]+'\nSignal {}'.format(i))
    display(mean_and_std(splited_signal, how='slices'))
    



v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv
Signal 0


,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.006596,0.005387,-0.002656,1.291797,1.211353,1.248803
1,-0.018439,0.013387,0.009895,1.297516,1.211862,1.251460
2,-0.003781,0.016835,-0.007428,1.294966,1.204689,1.242886
3,-0.008667,0.004428,0.006676,1.294298,1.208683,1.251964
4,-0.012507,0.019011,-0.006736,1.296621,1.221636,1.248193
5,-0.005880,0.005140,0.005844,1.289117,1.209591,1.245168
6,-0.019246,0.018696,0.004523,1.296954,1.209336,1.245688
7,-0.002106,0.005028,-0.003909,1.296644,1.205311,1.241622
8,-0.018772,0.009773,0.010368,1.296123,1.211373,1.261640
9,-0.006412,0.014458,-0.006451,1.298409,1.217039,1.253787



v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv
Signal 1


,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.019607,0.000180,0.016367,3.102329,2.364768,2.491220
1,0.021068,-0.008769,-0.013510,3.137001,2.401433,2.458621
2,-0.021246,0.029853,-0.007461,3.173365,2.442722,2.427374
3,-0.026125,0.012501,0.015446,3.175759,2.465075,2.433599
4,0.014802,-0.012230,-0.003962,3.160092,2.406104,2.490935
5,-0.019872,0.030912,-0.012813,3.104776,2.359437,2.496525
6,-0.010395,-0.002854,0.022380,3.144198,2.402743,2.452965
7,0.018713,0.000457,-0.012248,3.159520,2.441553,2.423222
8,-0.027631,0.039087,-0.009204,3.172987,2.458775,2.438671
9,-0.015051,-0.017587,0.025213,3.160334,2.403777,2.488405



v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv
Signal 2


,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.009033,0.009121,0.001576,1.557126,1.293221,1.119889
1,-0.024040,0.018686,0.003087,1.548669,1.285848,1.106367
2,-0.004320,0.013443,-0.004654,1.568302,1.296482,1.104084
3,-0.006066,0.005207,0.003678,1.567200,1.305422,1.099161
4,-0.018729,0.015649,0.007681,1.561267,1.290381,1.129608
5,-0.004880,0.017815,-0.005135,1.550151,1.284158,1.114267
6,-0.007053,0.009974,-0.003980,1.558204,1.287793,1.107035
7,-0.005135,0.008126,0.005938,1.567012,1.298340,1.103462
8,-0.019887,0.018602,0.003568,1.572879,1.299555,1.104010
9,-0.008970,0.016138,0.001008,1.559266,1.293978,1.121942



v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv
Signal 3


,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.003557,0.000585,0.023484,2.919351,2.911360,2.974313
1,0.010119,-0.002085,-0.012611,2.914251,2.902616,2.968005
2,-0.010804,0.028090,-0.011863,2.868312,2.915630,2.991311
3,-0.001712,0.010483,-0.008576,2.910053,2.937509,2.999621
4,-0.012807,0.023190,0.000797,2.913992,2.931411,2.998357
5,-0.018223,0.016347,0.011661,2.912236,2.922718,2.982980
6,-0.004295,-0.006134,0.003843,2.920474,2.905475,2.965367
7,-0.017303,0.011821,0.004531,2.879288,2.908103,2.993400
8,0.005732,-0.008323,-0.005629,2.906551,2.940424,3.007666
9,0.003144,0.012956,-0.009337,2.921137,2.930577,2.985920



v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv
Signal 4


,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.021076,0.018248,0.017560,1.183281,1.187313,1.189311
1,0.002128,0.008411,-0.007983,1.182007,1.186990,1.212213
2,-0.018718,0.013453,0.003179,1.189179,1.190064,1.197162
3,-0.007364,0.002763,0.003456,1.188078,1.190066,1.200361
4,-0.007201,0.019940,-0.003817,1.188984,1.187610,1.194986
5,-0.014584,0.007047,0.009661,1.184633,1.186842,1.191803
6,-0.004488,0.018490,-0.005512,1.178886,1.189132,1.201354
7,-0.013375,0.012058,0.010086,1.182728,1.192259,1.197553
8,-0.000167,0.008798,-0.008314,1.184217,1.189929,1.196181
9,-0.018019,0.016901,0.005248,1.181296,1.189471,1.193264


##### È possível notar que temos a média do sinal por volta de zero por causa de sua característica periódica.

### b) Extração de atributos no sinal

- Utilize o código de `feature_extraction_signal` em: https://github.com/navarmn/feature_extraction_signal
- Use as classes `Fourier` e `HOS` de maneira adequada. Lembre do método `.transform()`.
- Na classe `Fourier`utilize os parâmetros: `fs=5000`; o valor de fundamental está na TAG do nome em `_FG5955_`significa `fundamental=59.55`. `harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7)`, mas sintam-se à vontade para buscar outras harmônicas;
- Os rótulos devem serem impostos de acordo com a descrição da tag feita cima. 
- Monte um dataframe que contenham os atributos em cada coluna e a útlima com o rótulo. Concatene os artibutos extraídos da `Current_R`, `Current_S` e `Current_T`.
- Deverá ser feito um dataframe para os atributos de Fourier e um para os atributos de HOS.

### Extracting fundamental frequencies


In [130]:
fund_freq = [] #fundamental frequencies

#simple string manipulation to extract a fundamental frequencies from the datafile's name
for file_name in DATAFILES:
    a = file_name.split('_')
    fr = a[2] if a[1] == 'NORMAL' else a[5] 
    fund_freq.append(float(fr[2:])/100)
print("fund_freq =",fund_freq)

fund_freq = [60.0, 45.0, 60.0, 45.0, 60.0]


In [146]:
data = splited_signals[0][0]
display(data['Current_R'])

0       0.366486
1       0.570073
2       1.028144
3      -0.206103
4       1.091765
          ...   
4995    1.155386
4996    0.799109
4997    0.977247
4998    0.595521
4999    0.735488
Name: Current_R, Length: 5000, dtype: float64

In [148]:
#Fourier feature extractor
ffe = feature_extraction.Fourier(fundamental=60.00, fs=5000.0, harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7))
out_fourier = ffe.transform(data['Current_R'])
display(out_fourier)

ValueError: zero-size array to reduction operation maximum which has no identity

In [149]:
data = joinDataframe(splited_signals[0])
#Fourier feature extractor
ffe = feature_extraction.Fourier(fundamental=60.00, fs=5000.0, harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7))
out_fourier = ffe.transform(data['Current_R'])
display(out_fourier)

{'fft': array([0.02048141, 0.00091731, 0.00135496, ..., 0.00351013, 0.00028702,
        0.00204926]),
 'frequency': array([0.00000000e+00, 1.00004000e-01, 2.00008000e-01, ...,
        2.49979999e+03, 2.49990000e+03, 2.50000000e+03]),
 'features': [0.0010947844590054336,
  0.0012768600727408254,
  0.0019181296168689654,
  0.002627893671480861,
  0.002227436337641893,
  0.0029189683271301317,
  0.002224739786148706],
 'features_vector': [0.5000200008000321,
  1.0000400016000641,
  1.6000640025601023,
  2.700108004320173,
  3.2001280051202046,
  5.400216008640346,
  7.6003040121604855]}

### c) Análise exploratória na base criada com os extratores de atributos
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
- Faça uma análise para cada uma das bases:
    1. Features criada com Fourier
    2. Features criada com HOS
    3. Combinação das duas features (Fourier + HOS). 
> Dica: dê atenção ao coeficiente de correlação.

In [47]:
# code here

# Feature Selection

## 1. Realizar um estudo de relevância de atributos utilzando as técnicas exibidas na aula.

Nesse momento utilize a base de dados `Clinical_data_09-09-19-processed.csv`. Uma breve descrição da base:
- Contém um histórico dos registros de pacientes acometidos com uma determinada patologia.
- Os registros são variados, vão desde resposta de questionários médicos de anamnese, por exemplo, "*É fumante nos últimos 5 anos?*", "*Faz uso de álcool constantemente?*", até resultados de exames clínicos como ECG e Ecocardiograma.
- O rótulo dessa base é a coluna `Óbito`.

### Carregue a base

In [48]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.

In [49]:
# code here

### b) Identicar qual a variáveis parecem influenciar no óbito dos pacientes :(
- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [50]:
# code here

## 2. QUESTÃO DESAFIO
- Essa etapa não entrará com atividade avaliativa, pois a base de dados a seguir não é tão bonita quanto parece 🤪. Há muitas etapas de pré-processamento, agrupmaento e mais importante de tudo, o usuário tem que fazer as duas tabelas `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv` conversarem entre si.
- Fique à vontade para tentar e para tirar dúvidas. 😊

Nesse momento utilize a base de dados `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv`. Uma breve descrição da base:
- Essa base foi retirada de um serviço de streaming de midia. Os atributos são relativos a perfis de consumo de usuários.
- O objetivo é realizar detecção de *churn* (https://resultadosdigitais.com.br/blog/o-que-e-churn/)
- Possui um registro de 17 semanas de uso e os rótulos estáo na tabela `user-status-after-shrink.csv`, que indicam se ao final do período o usuário cancelou e se manteve assinante.

In [51]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
> Exiba a matriz de correlação!

In [52]:
# code here

### b) Identicar qual a variáveis parecem influenciar no cancelamento da assinatura.

- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [53]:
# code here